In [1]:
import numpy as np
import scipy as sp
# %matplotlib inline
import matplotlib.pyplot as plt
from collections import defaultdict
import nltk
import re
import string

In [2]:
# open docs file and read its lines
with open("train.dat", "r") as fh:
    train_lines = fh.readlines()  

In [3]:
train_lables = []
train_sentences = []
for line in train_lines:
    lable, sentence = line.split('\t', 1)
    train_lables.append(lable)
    train_sentences.append(sentence)

train_lables = train_lables[:1000]
train_sentences = train_sentences[:1000]

In [4]:
test_sentences = []
with open("test.dat", "r") as fh:
    test_sentences = fh.readlines() 
    
test_sentences = test_sentences[:50]

In [5]:
stopwords = set(nltk.corpus.stopwords.words('english') + ['', '\n'])
stemmer = nltk.stem.porter.PorterStemmer()
lematizer = nltk.stem.WordNetLemmatizer()

idx = {}
reverse_idx = {}

def preprocess(sentences,training=False):
    global stopwords,stemmer,lematizer,idx
    processed = []
    tid = len(idx)
    for sentence in sentences:
        # convert to lowercase
        sentence = sentence.lower()
        
        # remove non alphabets
        sentence = re.sub("[^a-z\s']", ' ', sentence)

        #separate into words
        words = nltk.word_tokenize(sentence)
        
        # remove unnecessary english grammer words
        words = [word for word in words if word not in stopwords]

        # lematize and stem similar words
        words = [stemmer.stem(lematizer.lemmatize(word)) for word in words]
        
        if training:
            for w in words:
                if w not in idx:
                    idx[w] = tid
                    reverse_idx[tid] = w
                    tid += 1
        else:
            words = [word for word in words if word in idx]
                    
        processed.append(words)
    return processed

In [6]:
train_docs = preprocess(train_sentences,True)
test_docs = preprocess(test_sentences)

In [7]:
from nltk.probability import FreqDist

def most_common_unique(train_docs,train_lables,N=500):
    by_catogory_fdist=[None]*6
    by_catogory_most_common = [None]*6
    by_catogory_docs = [[],[],[],[],[],[]]
    by_catogory_most_common_unique = [None]*6

    for i in range(len(train_docs)):
        by_catogory_docs[(int(train_lables[i]))] += train_docs[i]
    for i in range(len(by_catogory_docs)):
        by_catogory_fdist[i]=FreqDist(nltk.Text(by_catogory_docs[i]))
#         by_catogory_fdist[i].plot(30)
        by_catogory_most_common[i] = [t[0] for t in by_catogory_fdist[i].most_common(N)]
    rest = []
    for i in range(1,len(by_catogory_most_common)):
        rest +=by_catogory_most_common[i]
    by_catogory_most_common_unique[0] = set(rest)
    for i in range(1,len(by_catogory_most_common)):
        rest = []
        for j in range(len(by_catogory_most_common)):
            if i != j:
                rest += by_catogory_most_common[j]
        rest = set(rest)
        by_catogory_most_common_unique[i] = set([w for w in by_catogory_most_common[i] if w not in rest])
#         print("Category:",i," \n",by_catogory_most_common_filtered[i])
#         print("")
    return by_catogory_most_common_unique

most_common_unique = most_common_unique(train_docs,train_lables)

def boost_frequency(docs,by_catogory_most_common_unique,lables=None, training = False):
    for i in range(len(docs)):
        # as most medical text words are greater than 10 chars boost the frequency of words having length more than 10
        length_boosted_words = [word for word in docs[i] if len(word)>10]
        
        # most common words in one category not occuring in most common of any other category get a uniqueness boost.
        uniqueness_boosted_words = []
        if training:
            uniqueness_boosted_words = [word for word in docs[i] if word in by_catogory_most_common_unique[int(lables[i])]]
        else:
            uniqueness_boosted_words = [word for word in docs[i] if word in by_catogory_most_common_unique[0]]
            
        docs[i] += length_boosted_words + uniqueness_boosted_words*2
    
    return docs
            
        
train_docs = boost_frequency(train_docs,most_common_unique,train_lables,True)
test_docs = boost_frequency(test_docs,most_common_unique)       

In [8]:
from collections import Counter
from scipy.sparse import csr_matrix



def build_matrix(docs):
    r""" Build sparse matrix from a list of documents, 
    each of which is a list of word/terms in the document.  
    """
    nrows = len(docs)
    global idx
    nnz = 0
    for d in docs:
        nnz += len(set(d))
    ncols = len(idx)
        
    # set up memory
    ind = np.zeros(nnz, dtype=np.int)
    val = np.zeros(nnz, dtype=np.double)
    ptr = np.zeros(nrows+1, dtype=np.int)
    i = 0  # document ID / row counter
    n = 0  # non-zero counter
    # transfer values
    for d in docs:
        cnt = Counter(d)
        keys = list(k for k,_ in cnt.most_common())
        l = len(keys)
        for j,k in enumerate(keys):
            ind[j+n] = idx[k]
            val[j+n] = cnt[k]*100/len(d)
        ptr[i+1] = ptr[i] + l
        n += l
        i += 1
            
    mat = csr_matrix((val, ind, ptr), shape=(nrows, ncols), dtype=np.double)
    mat.sort_indices()
    
    return mat

def csr_info(mat, name="", non_empy=False):
    r""" Print out info about this CSR matrix. If non_empy, 
    report number of non-empty rows and cols as well
    """
    if non_empy:
        print("%s [nrows %d (%d non-empty), ncols %d (%d non-empty), nnz %d]" % (
                name, mat.shape[0], 
                sum(1 if mat.indptr[i+1] > mat.indptr[i] else 0 
                for i in range(mat.shape[0])), 
                mat.shape[1], len(np.unique(mat.indices)), 
                len(mat.data)))
    else:
        print( "%s [nrows %d, ncols %d, nnz %d]" % (name, 
                mat.shape[0], mat.shape[1], len(mat.data)) )

In [9]:
# scale matrix and normalize its rows
def csr_idf(mat, copy=False, **kargs):
    r""" Scale a CSR matrix by idf. 
    Returns scaling factors as dict. If copy is True, 
    returns scaled matrix and scaling factors.
    """
    if copy is True:
        mat = mat.copy()
    nrows = mat.shape[0]
    nnz = mat.nnz
    ind, val, ptr = mat.indices, mat.data, mat.indptr
    # document frequency
    df = defaultdict(int)
    for i in ind:
        df[i] += 1
    # inverse document frequency
    for k,v in df.items():
        df[k] = np.log(nrows / float(v))  ## df turns to idf - reusing memory
    # scale by idf
    for i in range(0, nnz):
        val[i] *= df[ind[i]]
        
    return df if copy is False else mat

def csr_l2normalize(mat, copy=False, **kargs):
    r""" Normalize the rows of a CSR matrix by their L-2 norm. 
    If copy is True, returns a copy of the normalized matrix.
    """
    if copy is True:
        mat = mat.copy()
    nrows = mat.shape[0]
    nnz = mat.nnz
    ind, val, ptr = mat.indices, mat.data, mat.indptr
    # normalize
    for i in range(nrows):
        rsum = 0.0    
        prev_ind = -1
        curr_val = 0.0
        for j in range(ptr[i], ptr[i+1]):
            if ind[j] == prev_ind:
                curr_val += val[j]
            else:
                rsum += curr_val**2
                curr_val = val[j]
                prev_ind = ind[j]
        rsum += curr_val**2
        if rsum == 0.0:
            continue  # do not normalize empty rows
        rsum = 1.0/np.sqrt(rsum)
        for j in range(ptr[i], ptr[i+1]):
            val[j] *= rsum
            
    if copy is True:
        return mat

In [10]:
def find_weights(train_ground_truth):
    weight = []
    for l in range(1,6):
        weight.append(train_ground_truth.count(str(l)))
    
    minimum = float('inf')
    for l in weight:
        if 0<l<minimum:
            minimum=l
    for l in range(len(weight)):
        if weight[l]!=0:
            weight[l] = minimum/weight[l]
    return [0.0] + weight

def knn(k_nearest_indices,similarities,k,eplsilon,train_truths,weight):
    prediction = []
    for i in range(len(k_nearest_indices)):
        weighted_votes = [0.0]*6
        for j in range(min(len(k_nearest_indices[i]),k)):
            neighbour = k_nearest_indices[i][j]
            if eplsilon < similarities[i][neighbour]:
                weighted_votes[int(train_truths[neighbour])] += similarities[i][neighbour]

        max_vote = similarities[i][k_nearest_indices[i][0]]
        max_category = train_truths[k_nearest_indices[i][0]]
        for i in range(1,6):
            if weighted_votes[i]*weight[i] > max_vote:
                max_vote = weighted_votes[i]*weight[i]
                max_category = i

        prediction.append(str(max_category))
    return prediction

In [11]:
# performing 5 fold crossvalidation to find the best meta parameters fo k and epsilon
def five_fold_cross_validation(train_docs,train_lables):
    def f1_score(ground_truth,prediction):
        if len(ground_truth) != len(prediction):
            raise Exception("length is not same")

        TP = 0
        FN = 0
        TN = 0
        for i in range(len(ground_truth)):
            if ground_truth[i]==prediction[i]:
                TP+=1
            else:
                FN += 1

        return (2*TP/(2*TP+FN+TN))


    predictions=[]
    train_ground_truth = []
    test_ground_truth = []
    similarities = []
    k_nearest_indices =[]
    weights = []

    for i in range(5):
        train = train_docs[:int((i)*len(train_docs)/5)] + train_docs[int((i+1)*len(train_docs)/5):]
        test = train_docs[int((i)*len(train_docs)/5):int((i+1)*len(train_docs)/5)]
        
        
        mat_train = build_matrix(train)
        mat_test  = build_matrix(test)
        
        df = csr_idf(mat_train) #cf_idf applied to mat_train as it's passed by reference
        nnz = mat_test.nnz
        ind, val, ptr = mat_test.indices, mat_test.data, mat_test.indptr
        for idx in range(0, nnz):
            val[idx] *= df[ind[idx]] #cf_idf applied to mat_test
            
        csr_l2normalize(mat_train)
        csr_l2normalize(mat_test)
        
        similarities.append((mat_test.dot(mat_train.T)).toarray())
        k_nearest_indices.append((-similarities[i]).argsort())
        train_ground_truth.append(train_lables[:int((i)*len(train_docs)/5)] + train_lables[int((i+1)*len(train_docs)/5):])
        test_ground_truth.append(train_lables[int((i)*len(train_docs)/5):int((i+1)*len(train_docs)/5)])
        weights.append(find_weights(train_ground_truth[-1]))
    #     print(weight)


    final_k = 40
    final_epsilon = 0.60
    max_f1_score = 0.0

    for k in range(1,100):
        for e in range(0,100,5):
            epsilon = e/100
            score = 0
            for i in range(5):
                predictions.append(knn(k_nearest_indices[i],similarities[i],k,epsilon,train_ground_truth[i],weights[i]))
                score += f1_score(test_ground_truth[i],predictions[-1])

            if(score > max_f1_score):
                max_f1_score = score
                final_k = k
                final_epsilon = epsilon
    #         print(str(score/5) + ",", end='', flush=True)
    #     print(" ")
    
    return final_k,final_epsilon,max_f1_score/5
    


In [12]:
final_k,final_epsilon,expected_f1_score = five_fold_cross_validation(train_docs,train_lables)
print(final_k,final_epsilon,expected_f1_score)

86 0.0 0.7465763921156341


In [13]:
final_mat_train = build_matrix(train_docs)
final_mat_test  = build_matrix(test_docs)

df = csr_idf(final_mat_train) #cf_idf applied to mat_train as it's passed by reference
nnz = final_mat_test.nnz
ind, val, ptr = final_mat_test.indices, final_mat_test.data, final_mat_test.indptr
for idx in range(0, nnz):
    val[idx] *= df[ind[idx]] #cf_idf applied to mat_test

csr_l2normalize(final_mat_train)
csr_l2normalize(final_mat_test)

final_similarities = ((final_mat_test.dot(final_mat_train.T)).toarray())
final_k_nearest_indices = ((-final_similarities).argsort())
final_weights = find_weights(train_lables)
final_predictions = (knn(final_k_nearest_indices,final_similarities,final_k,final_epsilon,train_lables,final_weights))

print(final_mat_train.shape)
for p in final_predictions:
    print(p)

(1000, 8275)
3
5
2
2
1
3
4
4
1
3
5
4
2
2
1
1
3
1
4
4
1
1
4
4
1
1
1
4
1
1
3
2
2
1
4
1
3
5
5
1
4
4
4
4
4
4
3
4
1
1


In [14]:
# for sim in final_similarities:
#     for a in sim:
#         if a<0:
#             print(a)

In [15]:
# freq={}

# for doc in train_docs:
#     for word in doc:
#         if word not in freq:
#             freq[word] =0
#         freq[word] +=1

# for w in sorted(freq, key=freq.get, reverse=True):
#   print(w, freq[w])

In [16]:
# final_similarities.shape